In [1]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
with open('pl_fantasy.txt', 'rb') as f:
    text = f.read().decode('utf-8')

In [3]:
text = text.replace('\n\n', '\n')
print(len(text))
print(text[:250])

vocab = sorted(set(text))
len(vocab)

1917030
﻿ADRIAN PROŚCIAK

Cadmiium Weqquer

Świat niepokoju Czasem się zdaje, że życie w szczęściu, nie daje szczęścia, Czasem spokój, tak naprawdę nie daje spokoju… Wiec nie dziwię się tym, który porzucają szczęście by żyć na krawędzi, Bo życie choć moż


136

In [4]:
char2int = {letter: idx for idx, letter in enumerate(vocab)}
int2char = np.array(vocab)

text_int = [char2int[char] for char in text]

In [5]:
seq_len = 100
per_epoch = len(text) // (seq_len+1)

def split_input_target(chunk):
    x = chunk[:-1]
    y = chunk[1:]
    return x, y

batch_size = 64
sequences = tf.data.Dataset.from_tensor_slices(text_int).batch(seq_len+1, drop_remainder=True).map(split_input_target)
data = sequences.shuffle(10000).batch(batch_size, drop_remainder=True)
example_x, example_y = next(iter(data))

In [6]:
def translate(arr):
    print(''.join([int2char[x] for x in arr]))

In [7]:
def Model(vocab_size, embedding_dim, units, bs):
    inputs = layers.Input(shape=[None], batch_size=bs)
    embed = layers.Embedding(vocab_size, embedding_dim)(inputs)
    lstm1 = layers.LSTM(units, return_sequences=True, stateful=True,
                       recurrent_initializer='glorot_uniform', dropout=0.2)(embed)
    lstm2 = layers.LSTM(units, return_sequences=True, stateful=True,
                       recurrent_initializer='glorot_uniform', dropout=0.2)(lstm1)
    drop = layers.Dropout(0.2)(lstm2)
    flow = layers.Dense(vocab_size)(drop)
    
    model = tf.keras.Model(inputs, flow)
    return model

In [8]:
model = Model(len(vocab), 256, 1024, bs=batch_size)

In [150]:
model.load_weights("weights")

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()#learning_rate=3e-3)
model.compile(optimizer, loss_fn)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, None)]              0         
_________________________________________________________________
embedding (Embedding)        (64, None, 256)           34816     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dropout (Dropout)            (64, None, 1024)          0         
_________________________________________________________________
dense (Dense)                (64, None, 136)           139400    
Total params: 13,813,896
Trainable params: 13,813,896
Non-trainable params: 0
_________________________________________________

In [10]:
history = model.fit(data, epochs=50, verbose=2)

Epoch 1/50
296/296 - 42s - loss: 2.5883
Epoch 2/50


KeyboardInterrupt: 

In [9]:
# model.save_weights("weights")
model = Model(len(vocab), 256, 1024, bs=1)
model.load_weights("weights")

In [10]:
num_generate = 1000

start = 'Pomyliłaś niebo z gwiazdami odbitymi nocą '
generated = [char2int[c] for c in start]
start = [char2int[c] for c in start]
start = tf.expand_dims(start, 0)
temperature = 1.0


flow = start
model.reset_states()
for num in range(num_generate):
    model_output = model(flow)

    distribution = model_output[:, -1, :] / temperature
    flow = tf.random.categorical(distribution, 1)
    
    sampled_letter = flow.numpy().squeeze()
    generated.append(sampled_letter)    

In [11]:
translate(generated)

Pomyliłaś niebo z gwiazdami odbitymi nocą i spokojnie wcześniej się nie spotkali. Zamiar zerwał z półkulą. Zaczął z nimi walczyć

- Żartujesz? - A tak

- Na wielką języku, teraz jeszcze wszystko dla niego jedza. Cztery nadruk wojskowy podbiegł do końca spalin i przypomniał sobie w kręgu głazy powoli, rozrywanej z ciemnego lampksa, ale bardzo potrafiłem zrobić twoją kawę i przekonać się, korytal pochodzić od czasu na rzeczywistości Jacka. Pomimo ciebie i początki sprawił, że usłyszała dźwięko. Powiedział Nagł się nieznacznie co za nami, potem razy, bo i jak widzę na tak zwanej przeciwnikami policjanci. – To na pewno. A teraz cała stara pokazać? - Ona się nie wydawało. Poza tym, dopiero pan gdzie ją mogli? – wyszeptał zmarł na słoneczne. - Jej negetynie pośpiesznie podbiegłem do niej i bez znajomą, jakby coś takiego. Poruszał się jakiś gwardzistów i drewnianych schodach jak ci ci dużo ciężki sprawę powiedzieli oszukać. A teraz byliśmy z Małą kapani jako zaś, zanim doszedł do samochodu 17